# **Part1. Text modeling using Berts**

## **1. Setting GPU, load data and bert embedding**

1.   parameters setting and GPU path sets
2.   load developement and temporal set
3.   tokenizer - embedding
4.   dataloader for training, val, test and temp



In [1]:
# check GPU
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-40GB


In [ ]:
path = '.../grace_icu/'   # change to the project path

import sys
import os

py_file_location = path
sys.path.append(os.path.abspath(py_file_location))

In [ ]:
# All of bert models' hyperparameters

para_all = {'note_type': 'physical_exam', # 'chief_complaint', 'history_of_present_illness', 'medications_on_admission', 'past_medical_history', 'physical_exam'
       'bert': 'clinical_longformer', # clinical_longformer, clinical_bigbird, bert, clinicalbert
       'seq_len': 240, # 60, 120, 240
       'token_max_length': 512, # 256, 512
       'batch_size': 16, # 12, 16
       'epochs': 2, # 1, 2, 3, 4
       'learning_rate': 1e-5} # 1e-5, 2e-5, 3e-5, 4e-5, 5e-5

# the name of saving
save_name = 'max_len-'+str(para_all['token_max_length'])+'_bs-'\
+str(para_all['batch_size'])+'_epoch-'+str(para_all['epochs'])+'_lr-'+str(para_all['learning_rate'])

# the result path for saving
result_path = path + 'result/subtext/' + para_all['note_type'] + '/' + para_all['bert'] + '/seq_' + str(para_all['seq_len']) + '/' + save_name + '/'
if not os.path.exists(result_path):
    os.makedirs(result_path)

print('para_all:', para_all)
print('save_name:', save_name)
print('result_path:', result_path)

In [ ]:
# install the related package
!pip install transformers

In [5]:
# upload the initial data and will be fed into the bert model - development & temporal & external dataset
import pandas as pd

# Load the dataset into a pandas dataframe: merge development and temporal set for easily processing
df, df_development, df_temporal, df_external = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
df_development = pd.read_csv(path+'data/seq_'+str(para_all['seq_len'])+'/subtext/development_text_older_'+para_all['note_type']+'.csv')
df_temporal = pd.read_csv(path+'data/seq_'+str(para_all['seq_len'])+'/subtext/temporal_text_older_'+para_all['note_type']+'.csv')
df_external = pd.read_csv(path+'data/seq_'+str(para_all['seq_len'])+'/subtext/external_text_older_'+para_all['note_type']+'.csv')
df = pd.concat([df_development, df_temporal, df_external], ignore_index=True)

id_source_info, id_source_dev_info, id_source_temp_info, id_source_ext_info = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
id_source_dev_info = pd.DataFrame({'id':list(df_development['id'].unique()), 'source':['development'] * len(list(df_development['id'].unique()))})
id_source_temp_info = pd.DataFrame({'id':list(df_temporal['id'].unique()), 'source':['temporal'] * len(list(df_temporal['id'].unique()))})
id_source_ext_info = pd.DataFrame({'id':list(df_external['id'].unique()), 'source':['external'] * len(list(df_external['id'].unique()))})
id_source_info = pd.concat([id_source_dev_info, id_source_temp_info, id_source_ext_info], ignore_index=True)

# Report the number of sentences.
print('Number of development sentences: {:,}\n'.format(df_development.shape[0]))
print('Number of temporal sentences: {:,}\n'.format(df_temporal.shape[0]))
print('Number of external sentences: {:,}\n'.format(df_external.shape[0]))
print('Number of total sentences: {:,}\n'.format(df.shape[0]))
print('Number of total patients: {:,}\n'.format(id_source_info.shape[0]))
# Display 5 random rows from the data.
df.sample(5)

del df_development, df_temporal, df_external, id_source_dev_info, id_source_temp_info, id_source_ext_info

Number of development sentences: 30,198

Number of temporal sentences: 2,824

Number of external sentences: 3,880

Number of total sentences: 36,902

Number of total patients: 34,837



In [ ]:
# rename for the next step of embedding
df.rename(columns={'text': 'sentence'}, inplace=True)

df = df[['id', 'sentence', 'label']]

df.head(5)

In [7]:
# Get the lists of sentences and their labels.
sentences = df.sentence.values
labels = df.label.values
id_all = df.id.values

In [ ]:
# upload the tokenizer
print('Loading BERT tokenizer...')

from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertTokenizer

if para_all['bert'] == 'clinicalbert':
  tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
elif para_all['bert'] == 'clinical_longformer':
  tokenizer = AutoTokenizer.from_pretrained("yikuan8/Clinical-Longformer")
elif para_all['bert'] == 'clinical_bigbird':
  tokenizer = AutoTokenizer.from_pretrained("yikuan8/Clinical-BigBird")
else:
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# Sample case
# original sentence case
print(' Original: ', sentences[0])

# output the content after tokenization
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
print(len(tokenizer.tokenize(sentences[0])))

# map each word to the dictionary index.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

In [ ]:
# obtain the maximum and median length of tokens for the entire dataset
import statistics

max_len = 0
len_list = []
for sent in sentences:

    # tokenize the text and add [CLS] and [SEP] symbols
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
    len_list = len_list + [len(input_ids)]

print('Max sentence length: ', max_len)
print('Median sentence length: ', statistics.median(len_list))

In [ ]:
# store the tokenized dataset into a list after tokenization
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # note
                        add_special_tokens = True,           # add [CLS] and [SEP] symbols
                        max_length = para_all['token_max_length'],    # Padding & Truncation Length
                        pad_to_max_length = True,
                        return_attention_mask = True,          # return attn. masks.
                        return_tensors = 'pt',             # return the pytorch tensor type of data
                   )

    # add the encoded text to the list
    input_ids.append(encoded_dict['input_ids'])

    # add the text's attention mask to the attention_masks list as well.
    attention_masks.append(encoded_dict['attention_mask'])

# convert list to tensor
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
id_all = torch.tensor(id_all)

print('input_id shape: ', input_ids.shape)
print('attention_masks shape: ', attention_masks.shape)
print('labels shape: ', labels.shape)
print('id_all shape: ', id_all.shape)

In [ ]:
# output the original and encoded information of the text in the 1st row
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])
print('Attention masks:', attention_masks[0])

In [13]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset

def get_tensordataset(input_ids, attention_masks, labels, id_all, id_source_info):
  # id_source_info: By default, there is no overlap in IDs between development, temporal, and external; otherwise, the following content needs to be modified
  # tensor to numpy
  input_ids_new = input_ids.numpy()
  attention_masks_new = attention_masks.numpy()
  labels_new = labels.numpy()
  id_all_new = id_all.numpy()

  # get train, val, test, temp, ext id
  id_unique, id_train, id_val_test, id_val, id_test, id_temp, id_ext = [], [], [], [], [], [], []
  id_temp = id_source_info.loc[id_source_info['source'] == 'temporal','id'].values
  id_ext = id_source_info.loc[id_source_info['source'] == 'external','id'].values
  id_unique = id_source_info.loc[id_source_info['source'] == 'development','id'].values # get development set
  id_train, id_val_test = train_test_split(id_unique, test_size=0.2, random_state=0, shuffle=True)
  id_val, id_test = train_test_split(id_val_test, test_size=0.5, random_state=0, shuffle=True)
  # print(len(id_train), len(id_val), len(id_test))
  del id_val_test, id_unique
  id_info = {}
  id_info = {'train':id_train, 'val':id_val, 'test':id_test, 'temp': id_temp, 'ext': id_ext}
  print("train {} val {} test {} temp {} ext {}".format(len(id_train), len(id_val), len(id_test), len(id_temp), len(id_ext)))

  # get dataset
  dataset_development = {}
  for db_type in ['train', 'val', 'test', 'temp', 'ext']:
    pos = []
    input_ids_need, attention_masks_need, labels_need, id_info_need = np.array([]), np.array([]), np.array([]), np.array([])
    pos = [i for i, x in enumerate(id_all_new) if x in id_info[db_type]]
    input_ids_need = torch.from_numpy(input_ids_new[pos,:])
    attention_masks_need = torch.from_numpy(attention_masks_new[pos,:])
    labels_need = torch.from_numpy(labels_new[pos])
    id_info_need = torch.from_numpy(id_all_new[pos])
    print("{} with input_id shape {} and attention_mask shape {} and label shape {} and id_info shape {}".\
          format(db_type, input_ids_need.shape, attention_masks_need.shape, labels_need.shape, id_info_need.shape))
    dataset = TensorDataset(input_ids_need, attention_masks_need, labels_need, id_info_need)
    dataset_development[db_type] = dataset

  return dataset_development, id_info

In [ ]:
dataset_development, id_info = get_tensordataset(input_ids, attention_masks, labels, id_all, id_source_info)

In [ ]:
# save the train, val, test, temp, and ext id for extra analysis
id_info = pd.DataFrame.from_dict(id_info, orient='index').transpose()

id_info['train'] = id_info['train'].astype('Int64')
id_info['val'] = id_info['val'].astype('Int64')
id_info['test'] = id_info['test'].astype('Int64')
id_info['temp'] = id_info['temp'].astype('Int64')
id_info['ext'] = id_info['ext'].astype('Int64')

id_info.to_csv(result_path + 'id_info.csv', index=False)
id_info.head(10)

In [16]:
# acquire the dataloader of training, validation, test, and temp sets
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# For the fine-tune training，the batch size will be better to set as 16 or 32
batch_size = para_all['batch_size']

# Create Dataloaders for the training and validation sets, and randomly shuffle the training samples
train_dataloader = DataLoader(
            dataset_development['train'],  # training data
            sampler = RandomSampler(dataset_development['train']), # Random mini-batches
            batch_size = para_all['batch_size'] # Train in mini-batches
        )

# The validation set does not need to be randomized; sequential reading is fine here.
validation_dataloader = DataLoader(
            dataset_development['val'], # internal validation of the val data
            sampler = SequentialSampler(dataset_development['val']), # Sequentially select mini-batches
            batch_size = para_all['batch_size']
        )

# The validation set does not need to be randomized; sequential reading is fine here.
test_dataloader = DataLoader(
            dataset_development['test'], # internal validation of the test data
            sampler = SequentialSampler(dataset_development['test']), # Sequentially select mini-batches
            batch_size = para_all['batch_size']
        )

# The validation set does not need to be randomized; sequential reading is fine here.
temp_dataloader = DataLoader(
            dataset_development['temp'], # temporal validation data
            sampler = SequentialSampler(dataset_development['temp']), # Sequentially select mini-batches
            batch_size = para_all['batch_size']
        )

# The validation set does not need to be randomized; sequential reading is fine here.
ext_dataloader = DataLoader(
            dataset_development['ext'], # external validation data
            sampler = SequentialSampler(dataset_development['ext']), # Sequentially select mini-batches
            batch_size = para_all['batch_size']
        )

In [17]:
del input_ids, attention_masks, labels, id_all, sentences, df

## **2. Bert modeling**



1.   loading bert model
2.   fine tuning bert model for mortality predicting
3.   save train, val, test, temp, and ext all useful results



In [ ]:
from transformers import AutoModelForSequenceClassification, AdamW, AutoConfig,AutoTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig

if para_all['bert'] == 'clinicalbert':
  model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=2)
elif para_all['bert'] == 'clinical_longformer':
  model = AutoModelForSequenceClassification.from_pretrained("yikuan8/Clinical-Longformer", num_labels=2)
elif para_all['bert'] == 'clinical_bigbird':
  model = AutoModelForSequenceClassification.from_pretrained("yikuan8/Clinical-BigBird", num_labels=2)
else:
  model = BertForSequenceClassification.from_pretrained(
      "bert-base-uncased", # A 12-layer pretrained model with lowercased text
      num_labels = 2, # Number of classes -- 2 indicates binary classification
                      # You can change this number for multi-class classification tasks
      output_attentions = False, # Whether the model returns attention weights
      output_hidden_states = False, # Whether the model returns all hidden states
  )

# Run the model on the GPU
model.cuda()

In [ ]:
# Convert all model parameters into a list.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
from transformers import AdamW, BertConfig

# I believe 'W' stands for 'Weight Decay Fix'
optimizer = AdamW(model.parameters(),
                  lr = para_all['learning_rate'], # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )

from transformers import get_linear_schedule_with_warmup

# Number of training epochs. BERT authors suggest between 2 and 4, setting it too high can lead to overfitting
epochs = para_all['epochs'] # Preliminary experimental results show that epoch = 2 is sufficient

# Total number of training samples
total_steps = len(train_dataloader) * epochs

# Create a learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [21]:
import numpy as np

# Calculate accuracy based on prediction results and label data
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

torch.cuda.empty_cache()
import gc
gc.collect()
torch.cuda.memory_summary(device=None, abbreviated=False)

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        #   [3]: patient id info
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # print(b_input_ids.shape, b_input_mask.shape, b_labels.shape)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        # loss, logits = model(b_input_ids,
        #                      token_type_ids=None,
        #                      attention_mask=b_input_mask,
        #                      labels=b_labels)
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = output.loss
        logits = output.logits
        # print(loss.shape, logits.shape)
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            # (loss, logits) = model(b_input_ids,
            #                        token_type_ids=None,
            #                        attention_mask=b_input_mask,
            #                        labels=b_labels)
            output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = output.loss
            logits = output.logits

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
import pandas as pd

# Retain 2 decimal places
# pd.set_option('precision', 2)
pd.options.display.precision=3

# Load training statistics into a DataFrame
df_stats = pd.DataFrame(data=training_stats)

# Use the epoch value as the index for each row
df_stats = df_stats.set_index('epoch')

# Display the table data
df_stats

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sns
plt.rcParams['figure.dpi'] = 600
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['figure.figsize'] = (5, 4.3)
# plot style
# sns.set(style='darkgrid')

# Increase the plot size and font size.
# sns.set(font_scale=1.5)
# plt.rcParams["figure.figsize"] = (6,3)

# plot learning curves
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.savefig(result_path + 'train_val_loss.png')

plt.show()

In [ ]:
# Save the trained model and vocabulary
import os

# Path where the model is stored
output_dir = result_path + 'model_save/'

# Create the directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# save the model
torch.save(model, output_dir + 'model')
# load the model
# model = torch.load(output_dir + 'model')

del output_dir

In [ ]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []
    all_labels = []
    all_patient_ids = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        # b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        patient_ids = batch[3].to(device)
        # Compute logits
        with torch.no_grad():
            # logits = model(b_input_ids, b_attn_mask)
            output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = output.loss
            logits = output.logits
        all_logits.append(logits)
        all_labels.append(b_labels)
        all_patient_ids.append(patient_ids)

    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    all_patient_ids = torch.cat(all_patient_ids, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()
    all_labels = all_labels.cpu().numpy()
    all_patient_ids = all_patient_ids.cpu().numpy()

    # data_need
    data_need = {}
    data_need = {'probs':probs, 'all_labels':all_labels, 'all_patient_ids':all_patient_ids}

    return data_need

In [ ]:
# acquire all evaluation results
result_train, result_val, result_test, result_temp, result_ext = {}, {}, {}, {}, {} # store the internal test, temporal test, and external test results
result_train = bert_predict(model, train_dataloader)
result_val = bert_predict(model, validation_dataloader)
result_test = bert_predict(model, test_dataloader)
result_temp = bert_predict(model, temp_dataloader)
result_ext = bert_predict(model, ext_dataloader)

In [ ]:
import json

result_all_save = {}
result_all_save = {'train': {'probs': result_train['probs'].tolist(), 'all_labels': result_train['all_labels'].tolist(), \
                'all_patient_ids': result_train['all_patient_ids'].tolist()}, \
           'val': {'probs': result_val['probs'].tolist(), 'all_labels': result_val['all_labels'].tolist(), \
                'all_patient_ids': result_val['all_patient_ids'].tolist()}, \
           'test': {'probs': result_test['probs'].tolist(), 'all_labels': result_test['all_labels'].tolist(), \
                'all_patient_ids': result_test['all_patient_ids'].tolist()}, \
           'temp': {'probs': result_temp['probs'].tolist(), 'all_labels': result_temp['all_labels'].tolist(), \
                'all_patient_ids': result_temp['all_patient_ids'].tolist()}, \
           'ext': {'probs': result_ext['probs'].tolist(), 'all_labels': result_ext['all_labels'].tolist(), \
                'all_patient_ids': result_ext['all_patient_ids'].tolist()}
          }

with open(result_path + 'text_all.json', 'w') as fp:
  json.dump(result_all_save, fp)

# with open(result_path + text_all.json', 'r') as fp:
#     result_all_save = json.load(fp)

## **3. Bert model evaluation**

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc

def evaluate_roc(probs, y_true):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """
    preds = probs[:, 1]
    fpr, tpr, threshold = roc_curve(y_true, preds)
    roc_auc = auc(fpr, tpr)
    print(f'AUC: {roc_auc:.4f}')

    # Get accuracy over the test set
    y_pred = np.where(preds >= 0.5, 1, 0)
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy*100:.2f}%')

    # Plot ROC AUC
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
evaluate_roc(result_train['probs'], result_train['all_labels'])

In [ ]:
evaluate_roc(result_val['probs'], result_val['all_labels'])

In [ ]:
evaluate_roc(result_test['probs'], result_test['all_labels'])

In [ ]:
evaluate_roc(result_temp['probs'], result_temp['all_labels'])

In [ ]:
evaluate_roc(result_ext['probs'], result_ext['all_labels'])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, precision_recall_curve, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, average_precision_score
from sklearn.metrics import brier_score_loss

def model_performance_params(data, data_pred_proba, ts_use, ts_value):
    """
    data: the truth label of target [array]
    data_pred_proba: predict probability of target with one columns [array]
    ts_use: 'True' or 'False' (if true, will use ts_value, else will not use ts_value) [Bool]
    ts_value: float value (if ts_use = 'True', will use it - input the value needed, or not use it)

    """
    fpr, tpr, thresholds_ROC = roc_curve(data, data_pred_proba)
    precision, recall, thresholds = precision_recall_curve(data, data_pred_proba)
    average_precision = average_precision_score(data, data_pred_proba)
    brier_score = brier_score_loss(data, data_pred_proba, pos_label=1)
    roc_auc = auc(fpr, tpr)

    threshold_final = []
    if ts_use == 'False':
        optimal_idx = []
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds_ROC[optimal_idx]
        sensitivity = tpr[optimal_idx]
        specificity = 1 - fpr[optimal_idx]
        data_pred = np.zeros(len(data_pred_proba))
        data_pred[data_pred_proba >= optimal_threshold] = 1
        threshold_final = optimal_threshold
    else:
        optimal_idx = []
        optimal_idx = np.max(np.where(thresholds_ROC >= ts_value))
        sensitivity = tpr[optimal_idx]
        specificity = 1 - fpr[optimal_idx]
        data_pred = np.zeros(len(data_pred_proba))
        data_pred[data_pred_proba >= ts_value] = 1
        threshold_final = ts_value

    tn, fp, fn, tp = confusion_matrix(data, data_pred).ravel()
    accuracy = accuracy_score(data, data_pred)
    F1 = f1_score(data, data_pred)  # not consider the imbalance, using 'binary' 2tp/(2tp+fp+fn)
    precision_c = tp/(tp+fp)

    parameters = {'auc': roc_auc, 'sensitivity': sensitivity, 'specificity': specificity, 'accuracy': accuracy,
                  'F1': F1, 'precision': precision_c, 'ap':average_precision, 'brier_score': brier_score, 'threshold': threshold_final}
    roc_plot_data = {'fpr_data': fpr, 'tpr_data': tpr}
    return parameters, roc_plot_data

In [ ]:
# result save to csv
# https://stackoverflow.com/questions/48909110/python-pandas-mean-and-sum-groupby-on-different-columns-at-the-same-time

result_train_db = pd.DataFrame()
result_train_db[['probs_0','probs_1']] = result_train['probs']
result_train_db['all_labels'] = result_train['all_labels']
result_train_db['all_patient_ids'] = result_train['all_patient_ids']
result_train_db = result_train_db.groupby('all_patient_ids').agg({'probs_0':'mean', 'probs_1':'mean', 'all_labels':'max'})
result_train_db = result_train_db.reset_index(level=['all_patient_ids'])
result_train_db['db_type'] = 'train'

result_val_db = pd.DataFrame()
result_val_db[['probs_0','probs_1']] = result_val['probs']
result_val_db['all_labels'] = result_val['all_labels']
result_val_db['all_patient_ids'] = result_val['all_patient_ids']
result_val_db = result_val_db.groupby('all_patient_ids').agg({'probs_0':'mean', 'probs_1':'mean', 'all_labels':'max'})
result_val_db = result_val_db.reset_index(level=['all_patient_ids'])
result_val_db['db_type'] = 'val'

result_test_db = pd.DataFrame()
result_test_db[['probs_0','probs_1']] = result_test['probs']
result_test_db['all_labels'] = result_test['all_labels']
result_test_db['all_patient_ids'] = result_test['all_patient_ids']
result_test_db = result_test_db.groupby('all_patient_ids').agg({'probs_0':'mean', 'probs_1':'mean', 'all_labels':'max'})
result_test_db = result_test_db.reset_index(level=['all_patient_ids'])
result_test_db['db_type'] = 'test'

result_temp_db = pd.DataFrame()
result_temp_db[['probs_0','probs_1']] = result_temp['probs']
result_temp_db['all_labels'] = result_temp['all_labels']
result_temp_db['all_patient_ids'] = result_temp['all_patient_ids']
result_temp_db = result_temp_db.groupby('all_patient_ids').agg({'probs_0':'mean', 'probs_1':'mean', 'all_labels':'max'})
result_temp_db = result_temp_db.reset_index(level=['all_patient_ids'])
result_temp_db['db_type'] = 'temp'

result_ext_db = pd.DataFrame()
result_ext_db[['probs_0','probs_1']] = result_ext['probs']
result_ext_db['all_labels'] = result_ext['all_labels']
result_ext_db['all_patient_ids'] = result_ext['all_patient_ids']
result_ext_db = result_ext_db.groupby('all_patient_ids').agg({'probs_0':'mean', 'probs_1':'mean', 'all_labels':'max'})
result_ext_db = result_ext_db.reset_index(level=['all_patient_ids'])
result_ext_db['db_type'] = 'ext'

result_all_db = pd.DataFrame()
result_all_db = pd.concat([result_train_db, result_val_db, result_test_db, result_temp_db, result_ext_db], ignore_index=True)
result_all_db.to_csv(result_path + 'text_all.csv', index=False)

result_all_db.head(5)

In [ ]:
# save performance metric
result_train, result_val, result_test, result_temp, result_ext = [], [], [], [], []
para, roc_plot = model_performance_params(result_train_db['all_labels'], result_train_db['probs_1'], 'False', 0)
result_train = [round(para['auc'],3), round(para['sensitivity'],3), round(para['specificity'],3), round(para['accuracy'],3),
        round(para['F1'],3), round(para['precision'],3), round(para['ap'],3),
        round(para['brier_score'],3), para['threshold'], 'train']

para, roc_plot = model_performance_params(result_val_db['all_labels'], result_val_db['probs_1'], 'False', 0)
result_val = [round(para['auc'],3), round(para['sensitivity'],3), round(para['specificity'],3), round(para['accuracy'],3),
        round(para['F1'],3), round(para['precision'],3), round(para['ap'],3),
        round(para['brier_score'],3), para['threshold'], 'val']

para, roc_plot = model_performance_params(result_test_db['all_labels'], result_test_db['probs_1'], 'False', 0)
result_test = [round(para['auc'],3), round(para['sensitivity'],3), round(para['specificity'],3), round(para['accuracy'],3),
        round(para['F1'],3), round(para['precision'],3), round(para['ap'],3),
        round(para['brier_score'],3), para['threshold'], 'test']

para, roc_plot = model_performance_params(result_temp_db['all_labels'], result_temp_db['probs_1'], 'False', 0)
result_temp = [round(para['auc'],3), round(para['sensitivity'],3), round(para['specificity'],3), round(para['accuracy'],3),
        round(para['F1'],3), round(para['precision'],3), round(para['ap'],3),
        round(para['brier_score'],3), para['threshold'], 'temp']

para, roc_plot = model_performance_params(result_ext_db['all_labels'], result_ext_db['probs_1'], 'False', 0)
result_ext = [round(para['auc'],3), round(para['sensitivity'],3), round(para['specificity'],3), round(para['accuracy'],3),
        round(para['F1'],3), round(para['precision'],3), round(para['ap'],3),
        round(para['brier_score'],3), para['threshold'], 'ext']

result_all = pd.DataFrame()
result_all = pd.DataFrame([result_train, result_val, result_test, result_temp, result_ext], \
              columns=['auc', 'sensitivity', 'specificity', 'accuracy', 'F1', \
                  'precision', 'ap', 'brier_score', 'threshold', 'db_type']
              )
result_all.to_csv(result_path + 'text_metrics.csv', index=False)
print(result_all)

del result_train, result_val, result_test, result_temp, result_ext